In [24]:
from selenium import webdriver
import datetime, re, requests, io, time, random, string
from bs4 import BeautifulSoup
from credentials import email, password
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [25]:
urls = {
    'sign_in': 'https://wallmine.com/users/sign-in',
    'homepage': 'https://wallmine.com/',
    'view_stock': lambda exchange, symbol: f'https://wallmine.com/{exchange}/{symbol}'
}
urls['view_stock']('nasdaq','tsla')

'https://wallmine.com/nasdaq/tsla'

In [26]:
driver = webdriver.Chrome('/Users/nikkisalehi/downloads/chromedriver')

In [28]:
def login():
    '''Method used to login into Wallmine account'''
    driver.get('https://wallmine.com/users/sign-in')
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click()
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
    time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
    time.sleep(3)
    if driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/span'):
        print('Login was successful')
        

In [29]:
login()

Login was successful
